<a href="https://colab.research.google.com/github/umutcankarakas/lstmmusicgenerator/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/umutcankarakas/theorygenerator.git

Cloning into 'theorygenerator'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 51 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [ ]:
!pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.46.0-cp36-none-any.whl size=17278 sha256=e1a611c175f3c700e46eb019790ba436dfbdb6796a4430575305386430b6f047
  Stored in directory: /root/.cache/pip/wheels/d2/2e/80/fec4c05eb23c8e13b790e26d207d6e0ffe8013fad8c6bdd4d2
Successfully built keras-self-attention


In [ ]:
import os
import numpy as np
from pathlib import *
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
#from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

import string

Using TensorFlow backend.


In [ ]:
os.chdir('theorygenerator')

In [ ]:
train_path = '../drive/My Drive/ragtime/'

In [ ]:
def convertMidi2Arr(midi_element):
  newlist = []
  max_duration = 8*4*4
  if isinstance(midi_element, note.Note):
    newlist.append(midi_element.pitch.midi)
    if int(round(midi_element.duration.quarterLength*4)) > max_duration:
      newlist.append(128 + max_duration)
    else:
      newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  elif isinstance(midi_element, chord.Chord):
    for n in midi_element:
      newlist.append(n.pitch.midi) 
      if int(round(midi_element.duration.quarterLength*4)) > max_duration:
        newlist.append(128 + max_duration)
      else:
        newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  elif isinstance(midi_element, note.Rest):
    newlist.append(128)
    if int(round(midi_element.duration.quarterLength*4)) > max_duration:
      newlist.append(128 + max_duration)
    else:
      newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  return newlist

In [ ]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('../drive/My Drive/data/notes_attention_drop', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    #pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = 128 + 128 + 2

    network_input, normalized_input = prepare_sequences(notes, n_vocab)
    model = create_network_predict(normalized_input, n_vocab)
   
    prediction_output = generate_notes(model, network_input, n_vocab)
    create_midi(prediction_output)
    return prediction_output

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 400

    # get all pitch names
    #pitchnames = sorted(set(item for item in notes))
    #pitchnames.remove('_')

     # create a dictionary to map pitches to integers
    #note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        if '_' in sequence_in:
          i = sequence_in.index('_') + 1
          continue
        elif sequence_out == '_':
          i = i + sequence_length +1
          continue
        
        network_input.append([element for element in sequence_in])
        network_output.append(sequence_out)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)



In [ ]:

def create_network_predict(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()

    model.add(Bidirectional(LSTM(512,return_sequences=True),input_shape=(network_input.shape[1], network_input.shape[2]))) #n_time_steps, n_features? Needed input_shape in first layer, which is Bid not LSTM
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    
    model.add(LSTM(512,return_sequences=True))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('../drive/My Drive/checkpoints_attention_drop/weights-200-0.0272.hdf5')
    
    return model

In [ ]:
def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    #start = numpy.random.randint(0, len(network_input)-1)
    start = 0

    pattern = network_input[start]
    prediction_output = []
    prediction_output.extend(pattern)
    #prediction_output = pattern

    #C=    [0,12,24,36,48,60,72,84,96,108,120]
    #Cs =  [1,13,25,37,49,61,73,85,97,109,121]
    #D =   [2,14,26,38,50,62,74,86,98,110,122]
    #Ds =  [3,15,27,39,51,63,75,87,99,111,123]
    #E =   [4,16,28,40,52,64,76,88,100,112,124]
    #F =   [5,17,29,41,53,65,77,89,101,113,125]
    #Fs =  [6,18,30,42,54,66,78,90,102,114,126]
    #G =   [7,19,31,43,55,67,79,91,103,115,127]
    #Gs =  [8,20,32,44,56,68,80,92,104,116]
    #A =   [9,21,33,45,57,69,81,93,105,117]
    #As =  [10,22,34,46,58,70,82,94,106,118]
    #B =   [11,23,35,47,59,71,83,95,107,119]

    #filterOut = Cs + Ds + Fs + Gs + As

    # generate 500 notes
    for note_index in range(2000):
        print(note_index)
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        prediction[0][128] = -np.inf

        if not prediction_output or (prediction_output[-1] > 128 and prediction_output[-1]<257):
          prediction[0][range(128,257)] = -np.inf
        elif prediction_output[-1]<128 or prediction_output[-1] == 257:
          prediction[0][range(0,128)] = -np.inf
          prediction[0][257] = -np.inf

        if len(prediction_output) > 1 and prediction_output[-2]==257:
          prediction[0][257] = -np.inf


        #prediction[0][filterOut] = -np.inf


        result = numpy.argmax(prediction)
        prediction_output.append(result)

        pattern.append(result)
        pattern = pattern[1:len(pattern)]
    if prediction_output[0] == 257: prediction_output = prediction_output[2:]
    return prediction_output

In [ ]:
def create_midi(prediction_output):
  output_notes = []
  same_offset_elements = []
  same_offset_duration_list = []
  offset = 0.0
  prediction_output.append(257)
  prediction_output.append(128)
  for pitch,duration in zip(prediction_output[0::2], prediction_output[1::2]):
    if pitch!=257 and pitch != 128:
      new_note = note.Note(pitch)
      new_note.duration.quarterLength = (duration-128)/4
      new_note.storedInstrument = instrument.Piano()
      new_note.offset = offset
      same_offset_elements.append(new_note)
      same_offset_duration_list.append((duration-128)/4)
    elif(pitch == 128):
      new_note = note.Rest()
      new_note.duration.quarterLength = (duration-128)/4
      new_note.storedInstrument = instrument.Piano()
      new_note.offset = offset
      output_notes.append(new_note)
      continue
    else:
      if not same_offset_elements:
        same_offset_elements = []
        same_offset_duration_list = []
        offset = offset + (duration-128)/4
      elif len(same_offset_elements) == 1:
        offset = offset + (duration-128)/4
        output_notes.append(same_offset_elements[0])
        same_offset_elements = []
        same_offset_duration_list = []
      else:
        duration_set = set(same_offset_duration_list)
        chord_notes = []
        for dur in duration_set:
          for note_samp in same_offset_elements:
            if note_samp.duration.quarterLength == dur:
              chord_notes.append(note_samp)
          if len(chord_notes) == 1:
            output_notes.append(chord_notes[0])
            chord_notes = []
          else:
            new_chord = chord.Chord(chord_notes)
            new_chord.offset = offset
            new_chord.duration.quarterLength = dur
            output_notes.append(new_chord)
            chord_notes = []
        offset = offset + (duration-128)/4
        same_offset_elements = []
        same_offset_duration_list = []

  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp='test_output.mid')
  return output_notes


In [ ]:
x = []
if __name__ == '__main__':
    x = generate();


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27